In [1]:
import sys    # 시스템
import os     # 시스템
import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리
from bs4 import BeautifulSoup     # html 데이터 전처리
import requests                   # http 요청 라이브러리
import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [15]:
# CSV 파일 불러오기

keyword = pd.read_csv('C:/Users/socsoft/Downloads/internship_kwj/socsoft/법제처키워드/(v)불공정_키워드(117).csv', encoding='utf-8')
keyword

,제목
0,"기초연금법 소상공인법 등 법제처, 문재인 정부 입법성과 발표"
1,"법제처, 문재인 정부 내 포용적 복지 K-방역 등 입법으로 견인했다"
2,"부동산 투기 물의 'LH', 청렴도 최하위"
3,“근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
4,"요소수, 월평균 판매량 10% 초과 보관시 매점매석(종합)"
...,...
112,"[단독] 청, 1조원대 공무원 복지포인트 과세 검토, 헌재 정부는 비과세 고수"
113,프랜차이즈 본부 ‘갑질’ 집중조사
114,"[불붙는 프랜차이즈 개혁]공정위, ‘가맹본부 갑질’ 조사 강화 가맹거래조사팀 신설"
115,"통일부 등 12개 부처 '우수' 평가 받아 ""한반도 평화체제 기반 마련"""


In [16]:
# 데이터 프레임 열에 이름 없을 경우 이름달아 주기

keyword=keyword.rename(columns={'Unnamed: 0':'순번','0':'제목'})

In [17]:
# 데이터 프레임에서 key 값으로 쓸 열을 리스트 형태로 가져오기
 
key = keyword["제목"]

key

0                  기초연금법 소상공인법 등 법제처, 문재인 정부 입법성과 발표
1              법제처, 문재인 정부 내 포용적 복지 K-방역 등 입법으로 견인했다
2                            부동산 투기 물의 'LH', 청렴도 최하위
3                       “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
4                   요소수, 월평균 판매량 10% 초과 보관시 매점매석(종합)
                           ...                      
112      [단독] 청, 1조원대 공무원 복지포인트 과세 검토, 헌재 정부는 비과세 고수
113                               프랜차이즈 본부 ‘갑질’ 집중조사
114    [불붙는 프랜차이즈 개혁]공정위, ‘가맹본부 갑질’ 조사 강화 가맹거래조사팀 신설
115         통일부 등 12개 부처 '우수' 평가 받아 "한반도 평화체제 기반 마련"
116                     '가계소득조사 개편 논란' 통계청, 업무평가 최하점
Name: 제목, Length: 117, dtype: object

In [19]:
# 결과 저장 코드

news_data = {}    # 전체 크롤링 데이터를 담을 그릇

# 네이버 인링크 추출
inlink_url = []

# 사이트 주소
print('네이버 뉴스 인링크 수집중...')
for i in tqdm_notebook(range(0, len(key))):
    try:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'.format(key[i])
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text)
        
        # 뉴스 전체 정보
        n_info = soup.find('a', class_='news_tit')

        # 네이버 인링크 정보
        onlink_info_1 = n_info['onclick'].split('&')[3]
        onlink_info_2 = onlink_info_1.split('=')[1]
        onlink_info = onlink_info_2.split('.')

        # 네이버 인링크 URL 리스트에 저장
        onlink_url = 'https://n.news.naver.com/mnews/article/{}/{}'.format(onlink_info[0], onlink_info[1])
        inlink_url.append(onlink_url)
    except:
        inlink_url.append('키워드: {}'.format(key[i]))        
print(len(inlink_url), '개 완료')
print('\n')

# 가져온 네이버 인링크 URL로 네이버뉴스 검색
print('인링크 뉴스데이터 수집중...')
for i in tqdm_notebook(range(0, len(key))):
    try:
        url = inlink_url[i]
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        resp = requests.get(url, headers=headers)       
        soup = BeautifulSoup(resp.text)
        
        target_info = {}  # 개별 뉴스 내용을 담을 딕셔너리 생성
        
        # 네이버 인링크 뉴스 정보
        news_info = soup.find('div', class_='newsct')

        # 제목 크롤링
        title = news_info.find('h2', 'media_end_head_headline').text

        # 글쓴이 크롤링
        try:
            nickname = news_info.find('em', class_='media_end_head_journalist_name').text
        except:
            nickname = 'None'
        # 날짜 크롤링
        datetime = news_info.find('span', class_='media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').text

        # 내용 크롤링
        contents = news_info.find('div', class_='newsct_article _article_body')
        content1 = contents.find(class_='go_trans _article_content')
        content = content1.text.replace('\n','')
        
        # 언론사 이름 크롤링
        company = news_info.find('img')['title']

        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['제목'] = title
        target_info['기자명'] = nickname
        target_info['작성일'] = datetime
        target_info['본문'] = content
        target_info['언론사'] = company
        target_info['url'] = inlink_url[i]
        
        # 각각의 글은 dict라는 딕셔너리에 담음
        news_data[i] = target_info
        
    # 에러날 경우 예외 처리
    except:
        #!time.sleep(0.5)
        continue
        
    if len(news_data) % 10 == 0:
        print(i, title)
        
print('수집한 글 갯수: ', len(news_data))

# 크롤링 결과 데이터 프레임으로 만들기
result_df = pd.DataFrame.from_dict(news_data, 'index')

# 데이터 프레임을 엑셀로 저장
result_df.to_excel("naver_news_content.xlsx", encoding='utf-8-sig')

네이버 뉴스 인링크 수집중...


C:\Users\socsoft\AppData\Local\Temp\ipykernel_11628\1943654342.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(key))):


  0%|          | 0/117 [00:00<?, ?it/s]

117 개 완료


인링크 뉴스데이터 수집중...


C:\Users\socsoft\AppData\Local\Temp\ipykernel_11628\1943654342.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(key))):


  0%|          | 0/117 [00:00<?, ?it/s]

16 법제처 "文정부 4년간 행정기본법 제정 등 성과"
34 정부·여당발 '플랫폼 규제법' 봇물…"反신산업법 될 수도"
49 이강섭 법제처장 “정부기관 법령입안 선제적으로 지원”
61 〔ANN의 뉴스 포커스〕15개 학·협회 단체가 모여 펼친 국가계약법 개정을 위한 담론의 장
79 대리점 단체 구성으로 협상력 제고… 본사 보복시 최대 3배 배상
95 양대노총 “공동 행정소송 제기”… ‘특별연장근로 확대 저지’ 나서
110 "취직했으니 대출금리 낮춰주세요"…'금리인하요구권' 입법예고
수집한 글 갯수:  76


In [ ]:
5290.0000151925&u='+urlencode(this.href));"

In [ ]:
032.0003097881&u='+urlencode(this.href));"

In [29]:
key[0:10]

0             기초연금법 소상공인법 등 법제처, 문재인 정부 입법성과 발표
1         법제처, 문재인 정부 내 포용적 복지 K-방역 등 입법으로 견인했다
2                       부동산 투기 물의 'LH', 청렴도 최하위
3                  “근로자인 조부모도 손자녀 육아휴직 근거 마련해야”
4              요소수, 월평균 판매량 10% 초과 보관시 매점매석(종합)
5         "연말까지 요소수 매점매석 금지" 어기면 '최대 3년' 징역 때린다
6                        ‘재난지원금 역풍’ 수습 나선 경북교육청
7                            성동구 ‘공정무역도시’ 공식 인증
8    "작전 막자" 전환가액 '상향 리픽싱' 도입 앞두고 CB 발행 1.8조 급증
9               중기부, 중소기업 권리구제 나선다 "기술유출 수사 강화"
Name: 제목, dtype: object